In [1]:
cd ..

/home/is/akiyoshi-n/my-project


In [2]:
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_dataset_2class_classification,load_dataset_4class_Multi_classification, split_test_data, split_data_by_index
from src.my_project.train_v2 import ActivityPrediction
from sklearn.model_selection import train_test_split
from src.my_project.dataset import load_multiclass_dataset
import wandb
import numpy as np

In [3]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))

In [4]:
# 最大トークン数
MAX_LEN = 128
# バッチサイズ
BATCH_SIZE = 16
# エポック数
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# Cross Validation時のFold数
NUM_FOLDS = 5
# 早期停止のための忍耐値
PATIENCE = 2
# 乱数シード
SEED = 2023
# クラス数
NUM_LABELS = 4
# 閾値
THRESH = 0.5

In [5]:
# データの読み込み
single_data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final.xlsx")

In [6]:
# 訓練データと評価データを辞書型で抽出
single_train_dataset = {
    'texts': [single_data['texts'][i] for i in range(900)],
    'labels': [single_data['labels'][i] for i in range(900)]
}
single_eval_dataset = {
    'texts': [single_data['texts'][i] for i in range(900, 1100)],
    'labels': [single_data['labels'][i] for i in range(900, 1100)]
}

In [7]:
# データの読み込み
multi_data, class_name = load_dataset_4class_Multi_classification(f"{DATASET_PATH}/act_classification_final.xlsx")

In [8]:
# 訓練データと評価データを辞書型で抽出
multi_train_dataset = {
    'texts': [multi_data['texts'][i] for i in range(900)],
    'labels': [multi_data['labels'][i] for i in range(900)]
}
multi_eval_dataset = {
    'texts': [multi_data['texts'][i] for i in range(900, 1100)],
    'labels': [multi_data['labels'][i] for i in range(900, 1100)]
}

In [9]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = ActivityPrediction(model_name = MODEL_NAME, seed=SEED, num_single_labels=2, num_multi_labels=3)

In [10]:
prediction = Classifier_model.train_and_predict(single_train_dataset, single_eval_dataset, multi_train_dataset, multi_eval_dataset, single_eval_dataset, multi_eval_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='2step')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.750700,0.712592,0.435000,0.454106
2,0.679500,0.672664,0.650000,0.426230
3,0.608600,0.621315,0.670000,0.541667
4,0.507200,0.643246,0.710000,0.539683
5,0.388800,0.591453,0.700000,0.605263
6,0.242500,0.738655,0.735000,0.582677
7,0.133800,0.835762,0.755000,0.625954


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/428 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.709200,0.676955,0.125000,0.311420
2,0.644000,0.616897,0.305556,0.197531
3,0.593400,0.606966,0.194444,0.133333
4,0.550300,0.581011,0.305556,0.183333
5,0.487900,0.523743,0.680556,0.529599
6,0.410600,0.483453,0.708333,0.652179
7,0.337600,0.440977,0.694444,0.675071
8,0.258500,0.411099,0.791667,0.780873
9,0.194100,0.383634,0.763889,0.733784
10,0.133100,0.381665,0.736111,0.717914


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [11]:
from sklearn.metrics import accuracy_score, f1_score

def calculate_accuracy_f1(y_true, y_pred):
    # 各サンプルに対してAccuracyを計算
    accuracy = accuracy_score(y_true, y_pred)

    # 各サンプルに対してF1値を計算（マルチクラスの場合は'average'パラメータを適切に設定する）
    f1 = f1_score(y_true, y_pred, average='macro')

    return accuracy, f1

In [12]:
a, b = calculate_accuracy_f1(multi_eval_dataset['labels'], prediction)

In [13]:
print(a)
print(b)

0.63
0.571134589598087


### Cross validation

In [14]:
single_data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final.xlsx")

In [15]:
# データの読み込み
multi_data, class_name = load_dataset_4class_Multi_classification(f"{DATASET_PATH}/act_classification_final.xlsx")

In [16]:
# testデータと訓練に使用するデータに分割
multi_data, multi_test_data, train_index, test_index = split_test_data(data=multi_data, test_size=0.1, SEED=SEED)

In [17]:
single_data, single_test_data = split_data_by_index(single_data, train_index, test_index)

In [18]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = ActivityPrediction(model_name = MODEL_NAME, seed=SEED, num_single_labels=2, num_multi_labels=3)

In [19]:
# Cross Validation
prediction = Classifier_model.cross_validation(single_data, multi_data, single_test_data, multi_test_data, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, NUM_FOLDS, output_dir, project_name='2step_cross_validation')

-----------------Fold: 1-----------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/791 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.720700,0.688076,0.577889,0.505882
2,0.680900,0.666264,0.572864,0.568528
3,0.613000,0.617315,0.653266,0.605714
4,0.524300,0.606678,0.673367,0.688995
5,0.406300,0.667379,0.678392,0.636364
6,0.302600,0.669783,0.698492,0.677419


...

Map:   0%|          | 0/359 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.717700,0.667672,0.164835,0.343793
2,0.661200,0.619967,0.307692,0.183486
3,0.604000,0.599563,0.318681,0.191358
4,0.564300,0.562161,0.406593,0.269453
5,0.508600,0.534575,0.571429,0.421356
6,0.443000,0.497101,0.670330,0.646529
7,0.369300,0.471300,0.670330,0.634519
8,0.321200,0.449957,0.681319,0.674728
9,0.228000,0.492371,0.703297,0.642045
10,0.169900,0.467162,0.670330,0.659779


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

{'accuracy': 0.5818181818181818, 'f1': 0.5702353561499377}


In [20]:
print(prediction)

[{'accuracy': 0.4818181818181818, 'f1': 0.51331569664903}, {'accuracy': 0.5636363636363636, 'f1': 0.5168800971716121}, {'accuracy': 0.5545454545454546, 'f1': 0.5435604566137984}, {'accuracy': 0.5727272727272728, 'f1': 0.5506162005785921}, {'accuracy': 0.5818181818181818, 'f1': 0.5702353561499377}]


In [21]:
average_accuracy = sum(d['accuracy'] for d in prediction)/len(prediction)
average_f1 = sum(d['f1'] for d in prediction)/len(prediction)
print("Average accuracy:", average_accuracy)
print("Average f1:", average_f1)

Average accuracy: 0.550909090909091
Average f1: 0.5389215614325941
